In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.metrics as metric
from math import sqrt
from prettytable import PrettyTable as PT

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

# if ram_gb < 20:
#   print('Not using a high-RAM runtime')
# else:
#   print('You are using a high-RAM runtime!')

In [ ]:
# pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#pip install scikit-surprise
import surprise
from surprise import KNNWithMeans
from surprise.model_selection import GridSearchCV
from surprise import Dataset
from surprise import accuracy
from surprise import Reader


In [ ]:
df = pd.read_json('Digital_Music.json', lines = True)

In [ ]:
# df.head(5)

In [ ]:
# df.describe(include='all')

In [ ]:
null_data = pd.DataFrame(df.isnull().mean() * 100)
null_data

,0
overall,0.000000
verified,0.000000
reviewTime,0.000000
reviewerID,0.000000
asin,0.000000
style,17.250875
reviewerName,0.005113
reviewText,0.091725
summary,0.033774
unixReviewTime,0.000000


In [ ]:
# df.reviewText.isnull().value_counts()

In [ ]:
new_df = df[df.reviewText.notna()]

In [ ]:
# new_df.shape

In [ ]:
after_df = new_df[['overall', 'unixReviewTime', 'reviewerID', 'asin','reviewText']]
after_df.rename(columns={'overall': 'ratings'}, inplace=True)


<ipython-input-13-18d15c50fa1d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_df.rename(columns={'overall': 'ratings'}, inplace=True)


In [ ]:
# after_df.head()

In [ ]:
after_df.drop('unixReviewTime', axis=1, inplace=True)

<ipython-input-15-141d4d997413>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after_df.drop('unixReviewTime', axis=1, inplace=True)


In [ ]:
# after_df.head()

In [ ]:
training_data, testing_data = train_test_split(
    after_df, test_size = 0.25, random_state = 42
)

In [ ]:
reviewer = after_df.groupby('reviewerID').count()
high_Rate_Reviewer = reviewer[reviewer.ratings >= 3].index
top_reviews_Df = after_df[after_df.reviewerID.isin(high_Rate_Reviewer)]

In [ ]:
asin = after_df.groupby('asin').count()
high_Rate_Asins = asin[asin.ratings >= 3].index
high_Asin = top_reviews_Df[top_reviews_Df.asin.isin(high_Rate_Asins)]


In [ ]:
# high_Asin.head()

In [ ]:
# converting data to surprise reader format
from surprise.model_selection import train_test_split
surprise_reader = Reader()
surprise_data = Dataset.load_from_df(
    high_Asin[['reviewerID', 'asin', 'ratings']],
    surprise_reader)
# Training the Model
training_set, testing_set = train_test_split(surprise_data, 
                                             test_size=0.25,
                                             random_state=42)


training_set.__class__, testing_set.__class__

(surprise.trainset.Trainset, list)

In [ ]:
from surprise import KNNBasic, SVD, NormalPredictor, KNNBaseline,KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering, Reader, dataset, accuracy
from collections import defaultdict
svd_model_50 = SVD(n_factors=50,reg_all=0.02)
svd_model_50.fit(training_set)

In [ ]:
svd_model_25 = SVD(n_factors=25,reg_all=0.02)
svd_model_25.fit(training_set)

In [ ]:
svd_model_75 = SVD(n_factors=75,reg_all=0.02)
svd_model_75.fit(training_set)

In [ ]:
svd_25_test_predictions = svd_model_25.test(testing_set)
svd_50_test_predictions = svd_model_50.test(testing_set)
svd_75_test_predictions = svd_model_75.test(testing_set)

In [ ]:
eval_table = PT()
# print(eval_table)
eval_table.field_names = ["Model Type", "RMSE Error"]

In [ ]:
eval_table.add_row(["SVD n_factors=25", accuracy.rmse(svd_25_test_predictions)])
eval_table.add_row(["SVD n_factors=50", accuracy.rmse(svd_50_test_predictions)])
eval_table.add_row(["SVD n_factors=75", accuracy.rmse(svd_75_test_predictions)])

RMSE: 0.6553
RMSE: 0.6544
RMSE: 0.6542


In [ ]:
print(eval_table)

+------------------+--------------------+
|    Model Type    |     RMSE Error     |
+------------------+--------------------+
| SVD n_factors=25 | 0.655305960618366  |
| SVD n_factors=50 | 0.6543662899455052 |
| SVD n_factors=75 | 0.6541507211028887 |
+------------------+--------------------+


In [ ]:
svd_param_grid = {
    'n_factors' : list(range(5, 100, 10)),
    'reg_all': [0.01, 0.02, 0.03]
}

In [ ]:
svd_grid_search = GridSearchCV(
    SVD,
    svd_param_grid,
    measures = ['rmse'],
    cv = 5,
    refit = True
)

In [ ]:
svd_grid_search.fit(surprise_data)

In [ ]:
svd_grid_search.best_params

{'rmse': {'n_factors': 65, 'reg_all': 0.01}}

In [ ]:
svd_cv_testing = svd_grid_search.test(testing_set)

In [ ]:
eval_table.add_row(["Best SVD n_factors=65", accuracy.rmse(svd_cv_testing)])
print(eval_table)

RMSE: 0.4804
+-----------------------+--------------------+
|       Model Type      |     RMSE Error     |
+-----------------------+--------------------+
|    SVD n_factors=25   | 0.655305960618366  |
|    SVD n_factors=50   | 0.6543662899455052 |
|    SVD n_factors=75   | 0.6541507211028887 |
| Best SVD n_factors=85 | 0.4803509867534198 |
| Best SVD n_factors=65 | 0.4803509867534198 |
+-----------------------+--------------------+
